<h1>Модуль profile и cProfile</h1>

Стандартные библиотеки Python поражают своим разнообразием. В них, кажется, есть всё, что только может понадобится разработчику, и профайлеры не исключение. На самом деле их целых три «из коробки»:

cProfile — относительно новый (с версии 2.5) модуль, написанный на C и оттого быстрый

profile — нативная реализация профайлера (написан на чистом питоне), медленный, и поэтому не рекомендуется к использованию

hotshot — экспериментальный модуль на си, очень быстрый, но больше не поддерживается и в любой момент может быть удалён из стандартных библиотек


Профилирование — сбор статистики во время работы программы. В дальнейшем будет рассмотрено использование памяти и измерение
времени работы.


Данные полученные в результате профилирования могут быть отформатированы в отчеты через модуль pstats - его более подробно рассмотрим позже.


cProfile - это детерминированный профилировщик: в нем отслеживаются вызов функции, возврат функции и события исключения, а также точные тайминги для интервалов между этими событиями (до 0,001 с). Документация библиотеки ([ https://docs.python.org/2/library/profile.html][1]) предоставляет нам простой пример использования


In [5]:
import hashlib
import cProfile
 
cProfile.run("hashlib.md5(b'abcdefghijkl').digest()")


         5 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method _hashlib.openssl_md5}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'digest' of '_hashlib.HASH' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




Здесь мы импортировали модуль hashlib и использовали cProfile для профилирования того, что создал хеш MD5. Первая строка показывает, что в ней 4 вызова функций. Следующая строка говорит нам, в каком порядке результаты выдачи. Но что же означают эти столбцы?

ncalls – это количество совершенных вызовов;

tottime – это все время, потраченное в данной функции;

percall – ссылается на коэффициент tottime, деленный на ncalls;

cumtime – совокупное время, потраченное как в данной функции, так и наследуемых функциях. Это работает также и с рекурсивными функциями!

Второй столбец percall – это коэффициент cumtime деленный на примитивные вызовы;

filename:lineno(function) предоставляет соответствующие данные о каждой функции.


Есть такое понятие - Примитивный вызов. Это вызов, который не был совершен при помощи рекурсии. Это очень интересный пример, так как здесь нет очевидных узких мест. Давайте создадим часть кода с узкими местами, и посмотрим, обнаружит ли их профайлер.

In [2]:
# -*- coding: utf-8 -*-
import time
import cProfile

def fast():
    print("Я быстрая функция")
 
 
def slow():
    time.sleep(3)
    print("Я очень медленная функция")
 
 
def medium():
    time.sleep(0.5)
    print("Я средняя функция...")
 
 
def main():
    fast()
    slow()
    medium()
 
if __name__ == '__main__':
    main()
    
    
cProfile.run('main()')

Я быстрая функция
Я очень медленная функция
Я средняя функция...
Я быстрая функция
Я очень медленная функция
Я средняя функция...
         106 function calls in 3.503 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.501    0.501 <ipython-input-2-25ee32f7a195>:14(medium)
        1    0.000    0.000    3.503    3.503 <ipython-input-2-25ee32f7a195>:19(main)
        1    0.000    0.000    0.001    0.001 <ipython-input-2-25ee32f7a195>:5(fast)
        1    0.000    0.000    3.001    3.001 <ipython-input-2-25ee32f7a195>:9(slow)
        1    0.001    0.001    3.503    3.503 <string>:1(<module>)
        8    0.000    0.000    0.003    0.000 iostream.py:197(schedule)
        6    0.000    0.000    0.000    0.000 iostream.py:310(_is_master_process)
        6    0.000    0.000    0.001    0.000 iostream.py:323(_schedule_flush)
        6    0.000    0.000    0.003    0.000 iostream.py:386(write)
        8

На этот раз мы видим, что у программы ушло 3.5 секунды на запуск. Если вы изучите результаты, то увидите, что cProfile выявил медленную функцию, которая тратит 3 секунды на запуск. Это и есть самая «слабая» часть основной функции. Обычно, когда вы обнаруживаете такие места, вы можете попытаться найти самый быстрый способ выполнения вашего кода, или прийти к выводу, что такая задержка приемлема. В этом примере, мы знаем, что лучший способ ускорить функцию, то убрать вызов time.sleep, или, по крайней мере, снизить продолжительность сна. Вы можете также вызвать cProfile в командной строке, вместо применения в интерпретаторе. Как это сделать:

In [9]:
python -m cProfile ptest.py


SyntaxError: invalid syntax (<ipython-input-9-472722e8767a>, line 1)

<h1>Выясняем скорость загрузки сайтов</h1>
Давайте немного развлечемся. С помощью модуля cProfile мы можем узнавать не только время выполнения команд в программе, но и время работы других запросов.
Сделаем небольшой марафон и узнаем какой сайт быстрее всех откроется из нашей программы.

In [12]:
# -*- coding: utf-8 -*-
import requests
import cProfile
 
 
def facebook():
    requests.get('https://facebook.com')
 
 
def google():
    requests.get('https://google.com')
	
	
def twitter():
    requests.get('https://twitter.com')
	
	
def vk():
    requests.get('https://vk.com')
 
 
def main():
    facebook()
    google()
    twitter()
    vk()
	
	
cProfile.run('main()')

         40197 function calls (40168 primitive calls) in 2.026 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.444    0.444 <ipython-input-12-fec7e32c8065>:10(google)
        1    0.000    0.000    0.519    0.519 <ipython-input-12-fec7e32c8065>:14(twitter)
        1    0.000    0.000    0.426    0.426 <ipython-input-12-fec7e32c8065>:18(vk)
        1    0.000    0.000    2.027    2.027 <ipython-input-12-fec7e32c8065>:22(main)
        1    0.000    0.000    0.638    0.638 <ipython-input-12-fec7e32c8065>:6(facebook)
        1    0.000    0.000    2.027    2.027 <string>:1(<module>)
        7    0.000    0.000    0.000    0.000 <string>:1(__new__)
       14    0.000    0.000    0.000    0.000 __init__.py:1412(debug)
       14    0.000    0.000    0.000    0.000 __init__.py:1677(isEnabledFor)
        7    0.000    0.000    0.002    0.000 _collections.py:140(__init__)
      118    0.001    0.000  

<h1>Справочник по модулю</h1>

Модули profile и cProfile обеспечивают следующие функции:

<b>profile.run(command, filename=None, sort=-1)</b>

Эта функция принимает один аргумент, который может быть передан функции exec(), и дополнительное имя файла. Во всех случаях эта подпрограмма выполняется:

In [13]:
exec(command, __main__.__dict__, __main__.__dict__)

NameError: name 'command' is not defined

Функция собирает статистику профилирования. Если имя файла не указано, то эта функция автоматически создает Stats сущность и печатает простой профильный отчет. Если вид, значение определены, то exec() передает Stats сущность, чтобы управлять, сортировкой результата.

<b>profile.runctx(command, globals, locals, filename=None, sort=-1)</b>

Эта функция похожа на run(), с добавленными аргументами, чтобы предоставить глобальные и локальные словари для command строк. 

In [1]:
exec(command, globals, locals)

NameError: name 'command' is not defined

Эта подпрограмма выполняется и собирает профильную статистику как в функции run() выше.

<b>class profile.Profile(timer=None, timeunit=0.0, subcalls=True, builtins=True)</b>

Этот класс обычно является используемый только в том случае, если требуется более точный контроль за профилированием, чем то, что обеспечивает функция cProfile.run().

Пользовательский таймер может быть предоставлен для измерения, сколько времени занимает код для запуска через аргумент timer. Это должна быть функция, которая возвращает одно число представляя текущее время. Если число является целым числом, timeunit определяет множитель, который указывает продолжительность каждой единицы времени. Например, если таймер возвращает время, измеренное в тысячах секунд, единица времени будет .001.

Непосредственно использование класса Profile позволяет форматировать результаты профиля, не в письме к данные о профиле файлу:

В версии Python 3.8  добавлена поддержка следующих функций:

enable()
Начать сбор данных профилирования. Только в cProfile.

disable()
Прекратить сбор данных профилирования. Только в cProfile.

create_stats()
Прекратить сбор данных профилирования и записать результаты в качестве текущего профиля.

print_stats(sort=-1)
Создайть объект Stats на основе текущего профиля и распечатать результаты для stdout.

dump_stats(filename)
Записать результаты текущего профиля в filename.

run(cmd)
Выполнить профилирование cmd через exec().

runctx(cmd, globals, locals)
Выполнить профилирование cmd через exec() с помощью указанной глобальной и локальной сред.

runcall(func, *args, **kwargs)
Профиль func(*args, **kwargs)

Следует отметить, что профилирование будет работать только в том случае, если вызываемая команда/функция действительно возвращает что-либо. Если интерпретатор завершается (например, посредством вызова sys.exit() во время выполнения вызываемой команды/функции), результаты профилирования не будут напечатаны.

<h1>Использование пользовательского таймера</h1>

Если необходимо изменить способ определения текущего времени (например, принудительно использовать время настенных часов или истекшее время процесса), передайте нужную функцию синхронизации конструктору класса Profile:

<b>pr = profile.Profile(your_time_func)</b>

Полученный профилировщик вызовет your_time_func. В зависимости от того, используется ли profile.Profile или cProfile.Profile, возвращает значение your_time_func будет интерпретироваться по-разному:

<b>profile.Profile</b>

your_time_func должен возвращает единственное число или список чисел, сумма которых - текущее время (как какой os.times() возвращает). Если функция возвращает один временной номер или список номеров возвращенный имеет длину 2, то вы получите особенно быструю версию подпрограммы отправки.

Обратите внимание на необходимость калибровки класса профилировщика для выбранной функции таймера (см. Ограничения). Для большинства машин, таймера, что возвращает одинокий целочисленный значение обеспечит лучшие результаты с точки зрения низкого наверху во время профилирования. (os.times() - pretty плохо как он возвращает кортеж плавающей запятой значения). Если вы хотите заменить лучший таймер самым чистым способом, выведите класс и выполните аппаратную проводку метода замены, который лучше всего обрабатывает ваш вызов таймера, вместе с соответствующей калибровочной константой.

<b>cProfile.Profile</b>

your_time_func должен возвращает один номер. Если это целые числа возвращает, вы можете также призвать конструктора класса со вторым аргументом, определяющим реальную продолжительность одной единицы времени. Например, если your_integer_time_func возвращает раз измерено в тысячах секунд, вы построите Profile сущность следующим образом:

<b>pr = cProfile.Profile(your_integer_time_func, 0.001)</b>

Поскольку класс cProfile.Profile не может быть откалиброван, пользовательские функции таймера должны быть используемый с осторожностью и должны быть как можно быстрее. Для лучших результатов с пользовательским таймером это могло бы быть необходимо для твердого-код это в источнике C внутреннего модуля _lsprof.